# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [1]:
class ActError(Exception):
    def __init__(self, msg=""):
        self.msg = msg
        
    def __str__(self):
        return 'Act "' + self.msg[:-1] + '" is not exist'

class PropertyCreator(type):
    def __new__(cls, name, bases, attr):
        
        g = 'get_'
        s = 'set_'
        d = 'del_'
        
        def isNoAct(msg):
            raise ActError(msg)
        
        vals = {}
        for key in attr:
            if key.startswith(g) or key.startswith(s) or key.startswith(d):
                valsName = key[len(g):]
                if valsName not in vals:
                    vals[valsName] = { g: lambda self: isNoAct(g),
                                       s: lambda self, value: isNoAct(s),
                                       d: lambda self: isNoAct(d) }
            if key.startswith(g):
                vals[valsName][g] = attr[key]
            if key.startswith(s):
                vals[valsName][s] = attr[key]
            if key.startswith(d):
                vals[valsName][d] = attr[key]
                
        for val in vals:
            attr[val] = property(vals[val][g], vals[val][s], vals[val][d])
        
        return super().__new__(cls, name, bases, attr)

#### Протестируйте свое решение

In [2]:
def mprint(*args, **kwargs):
    print('|', *args, **kwargs)

In [3]:
def test_simple():
    print('test_simple:')
    
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
            
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
    
    obj = TestPropertyCreator(lo=5)
    
    mprint(type(obj.x))
    
    try:
        obj.x = 4
    except ValueError as e:
        mprint(e)
    obj.x = 6
    mprint(type(obj.x))
    mprint(obj.x)
    del (obj.x)
    mprint(obj.x)
    
    print('OK')

def test_with_inheritance():
    print('test_with_inheritance:')
    
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")

    obj = TestPropertyCreatorInheritance()

    mprint(obj.x) #get
    obj.x = 6 #set
    mprint(obj.x) #get
    mprint(type(obj.x)) #get
    mprint(TestPropertyCreatorInheritance.x)
    mprint(type(TestPropertyCreatorInheritance.x))
    try:
        del(obj.x)
    except ActError as e:
        mprint(e)
    obj.x = 10 #set
    mprint(obj.x) #get
    try:
        mprint(TestPropertyCreator.x)
    except AttributeError as e:
        mprint(e)
    mprint(TestPropertyCreatorInheritance.x)
    mprint(obj._secret_list)
    
    print('OK')

def test_partially_defined():
    #Работу этого теста можно наблюдать в других
    print('test_partially_defined:')
    print('OK')

def test_sanity():
    print('test_sanity:')
    
    class TestPropertyCreator(metaclass=PropertyCreator):

        _text = 234325
        
        def get_raw_text(self):
             return self._boo

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))
                
    obj = TestPropertyCreator()
    mprint(dir(TestPropertyCreator))
    
    mprint(obj._text)
    mprint(type(obj._text))
    
    try:
        mprint(obj.raw_text)
    except AttributeError as e:
        mprint(e)
        
    mprint(obj.text)
    obj.text = 42
    
    mprint(obj._text)
    mprint(type(obj._text))
    mprint(obj.text)
    mprint(type(TestPropertyCreator.text))
    
    try:
        del(obj.text)
    except ActError as e:
        mprint(e)
    
    mprint('OK')

def test_multiple_usages():
    print('test_multiple_usages:')
    
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0

    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1

    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1

        def get_x(self):
            return self.value

    a = TestPropertyCreatorA()
    b = TestPropertyCreatorB()
    c = TestPropertyCreatorC()

    mprint(a.x)
    mprint(b.x)
    c.x = 12
    mprint(c.x)
    mprint(a.x)

    try:
        a.x = 10
    except ActError as e:
        mprint(e)
    try:
        del(a.x)
    except ActError as e:
        mprint(e)

    try:
        b.x = 10
    except ActError as e:
        mprint(e)
    try:
        del(b.x)
    except ActError as e:
        mprint(e)

    try:
        del(c.x)
    except ActError as e:
        mprint(e)
        
    print('OK')

test_simple()
print('\n')
test_with_inheritance()
print('\n')
test_sanity()
print('\n')
test_multiple_usages()

test_simple:
| <class 'NoneType'>
| Value must in condition: 5 <= value
| <class 'int'>
| 6
| No more
OK


test_with_inheritance:
| 0
| 0
| <class 'int'>
| <property object at 0x104b56f48>
| <class 'property'>
| Act "del" is not exist
| 0
| type object 'TestPropertyCreator' has no attribute 'x'
| <property object at 0x104b56f48>
| ['get', 'set', 'get', 'get', 'set', 'get']
OK


test_sanity:
| ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_text', 'get_raw_text', 'get_text', 'raw_text', 'set_text', 'text']
| 234325
| <class 'int'>
| 'TestPropertyCreator' object has no attribute '_boo'
| 1
| 42
| <class 'int'>
| 0
| <class 'property'>
| Act "del" is not exist
| OK


test_multiple_usages:
| 0
| 

## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
</code>

#### Решение

In [4]:
#Везде далее Exeption -> Exception

In [5]:
# Опишите исключение InstanceCountException
class InstanceCountException(Exception):
    def __init__(self, msg=""):
        self.msg = msg
        
    def __str__(self):
        return "Слишком много " + self.msg
        
try:
    raise InstanceCountException("I am here")
except InstanceCountException as e:
    print(e)

Слишком много I am here


In [6]:
# Опишите мета класс InstanceCountExceptioner
class InstanceCountExceptioner(type):
    
    __max_instance_count__ = 1
    
    def __new__(cls, name, bases, args):
        
        #Так как в условии этот момент не рассмотрен, то для каждого класса будет использоваться его изначальное значение,
        #но изменив его в ходе программы, можно сделать больше экземпляров этого класса.
        #Вариант, когда __max_instance_count__ меняется, но значение всегда будет только изначальное, реализовать не сложно, но зачем?
        if "__max_instance_count__" not in cls.__dict__:
            cls.bases["__max_instance_count__"] = InstanceCountExceptioner.__max_instance_count__
        
        return super().__new__(cls, name, bases, args)
    
    def __call__(cls, *args, **kwargs):
        if cls.__max_instance_count__ == 0:
            raise InstanceCountException(cls.__qualname__)
        cls.__max_instance_count__ -= 1        
        return super().__call__(*args, **kwargs) 

#### Протестируйте свое решение

In [7]:
class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 2

    def __init__(self):
        self.a = 1

    def get(self):
        return self.a

class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 3

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b
    
class TestInstanceCountExceptionerC(metaclass=InstanceCountExceptioner):

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b

def test():
    a_1 = TestInstanceCountExceptionerA()
    b_1 = TestInstanceCountExceptionerB()
    a_2 = TestInstanceCountExceptionerA()
    c_1 = TestInstanceCountExceptionerC() #Тут нет __max_instance_count__
    try:
        a_3 = TestInstanceCountExceptionerA()
    except InstanceCountException as e:
        print(e)
    try:
        a_4 = TestInstanceCountExceptionerA()
    except InstanceCountException as e:
        print(e)
    b_2 = TestInstanceCountExceptionerB()
    b_3 = TestInstanceCountExceptionerB()
    try:
        b_4 = TestInstanceCountExceptionerB()
    except InstanceCountException as e:
        print(e) 
    try:
        с_2 = TestInstanceCountExceptionerC()
    except InstanceCountException as e:
        print(e)

test()

Слишком много TestInstanceCountExceptionerA
Слишком много TestInstanceCountExceptionerA
Слишком много TestInstanceCountExceptionerB
Слишком много TestInstanceCountExceptionerC


## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [8]:
'''
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass
'''
pass

In [9]:
#print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

In [10]:
#tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [11]:
#tmp_obj = tmp()
#tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

In [12]:
#tmp.__dict__

#### Решение

In [13]:
import inspect
import json

class JSONClassCreator(type):
    def __new__(mcls, json_str):
        
        json_obj = json.loads(json_str)
        
        name = json_obj["name"]
        bases = tuple(map(lambda b: globals()[b], json_obj["bases"]))
        
        attrs = {}
        for key in json_obj["methods"]:
            attrs[key] = json_obj["methods"][key]
            exec(attrs[key])
            attrs[key] = locals()[key]
            
        for key in json_obj["attrs"]:
            attrs[key] = json_obj["attrs"][key]
        
        return super().__new__(mcls, name, bases, attrs)

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        
        methods = cls.__dict__.copy()
        attrs = {}
        
        for key in list(methods.keys()):
            if key in exclude:
                methods.pop(key)
                continue
            if not inspect.isfunction(methods[key]):
                attrs[key] = methods[key]
                methods.pop(key)
            else:
                code, _ = inspect.getsourcelines(methods[key])
                code = list(map(lambda x: x[code[0].index('d'):], code))
                methods[key] = "".join(code)

        return json.dumps({
            "name": cls.__name__,
            "bases": list(map(lambda b: b.__name__, cls.__bases__)),
            "methods": methods,
            "attrs": attrs
        })

    pass

#### Проверьте свое решение на примере

In [14]:
class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def ffff(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), "", sep="\n")

tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

print(tmp, type(tmp), "", sep="\n")

tmp_obj = tmp()
print(tmp_obj, tmp_obj.ffff("hi"), tmp.val, tmp.__doc__)
print(tmp.__dict__)

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'ffff': 'def ffff(self, x):\n    print(x)\n', '__repr__': 'def __repr__(self):\n    return "Test(val={})".format(self.val)\n', '__str__': 'def __str__(self):\n    return "Test(val={})".format(self.val)\n'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})

<class '__main__.Test'>
<class '__main__.JSONClassCreator'>

hi
Test(val=[1, 2, 3]) None [1, 2, 3] Тестовый класс
{'ffff': <function ffff at 0x104ba42f0>, '__repr__': <function __repr__ at 0x104ba4ae8>, '__str__': <function __str__ at 0x104ba4b70>, '__doc__': 'Тестовый класс', 'val': [1, 2, 3], '__module__': '__main__'}


In [15]:
code = """
def plus1(x):
    x = x + 1
    return x

class X():

    def __init__(self, a=42):
        self.a = a

    def f(self):
        print(self.a * 2)
        
x = X(22)
x.f()
print(plus1(22))

print('This is my output.')
"""

exec(code)

print(globals()['plus1'])

print(plus1(4))

44
23
This is my output.
<function plus1 at 0x104ba4158>
5
